In [1]:
import math
import gs_quant
from gs_quant.session import GsSession, Environment
from gs_quant.data import DataMeasure, DataFrequency
from gs_quant.markets.securities import SecurityMaster, AssetIdentifier
import datetime as dt
import pandas as pd


In [5]:
# setup client_id and client_secret for connection
GsSession.use(Environment.PROD, client_id, client_secret, 'read_product_data')

In [6]:
def get_vol_and_adv(ticker: str, start_date: dt.date, end_date: dt.date):
    """
    Fetch volatility and average daily trading volume for a stock.

    :param ticker: Stock ticker (e.g., 'AAPL')
    :param start_date: Start date for the data (e.g., dt.date(2023, 1, 1))
    :param end_date: End date for the data (e.g., dt.date(2023, 10, 1))
    :return: Dictionary with volatility and ADV/ADTV
    """
    # Get the asset using the ticker
    asset = SecurityMaster.get_asset(ticker, AssetIdentifier.TICKER)

    # Fetch historical close prices
    close_prices = asset.get_data_series(
        measure=DataMeasure.CLOSE_PRICE,
        frequency=DataFrequency.DAILY,
        start=start_date,
        end=end_date
    )

    # Calculate returns and volatility
    returns = close_prices.pct_change().dropna()
    volatility = returns.std()

    # Fetch historical trading volume
    trading_volume = asset.get_data_series(
        measure=DataMeasure.VOLUME,
        frequency=DataFrequency.DAILY,
        start=start_date,
        end=end_date
    )

    # Calculate average daily trading volume
    adv = trading_volume.mean()

    return volatility, adv

In [7]:
def lifetime_risk_tcm_bps( notional,
                           ticker,
                           date,
                           daily_vol = "Optional",
                           unwind_speed_as_pct_adv = 0.03):
    
    start_date = date.replace(year=date.year - 1)
    
    if daily_vol == "Optional":
        daily_vol, dollar_adv = get_vol_and_adv(ticker, start_date, date)
    else:
        dollar_adv = get_vol_and_adv(ticker, start_date, date)[1]
    days_to_unwind = abs( notional ) / ( unwind_speed_as_pct_adv * dollar_adv )
    lifetime_risk = math.sqrt( 1 / 3 ) * notional * daily_vol * math.sqrt( days_to_unwind )
    out = ( lifetime_risk / abs( notional ) ) * 10e3
    return( out, daily_vol )

In [8]:
notional = 100e6
# daily_vol = .05
# dollar_adv = 100e6
ticker = "AAPL"
date = dt.date(2025, 6, 9)


In [9]:
lifetime_risk_tcm_bps( notional, ticker, date )

(1594.1136536329607, 0.020907195709856222)

In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import datetime as dt

# Custom formatted text input widget
class FormattedTextInput:
    def __init__(self, description, initial_value=0):
        self.text_widget = widgets.Text(
            value=f"{initial_value:,}",
            description=description,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        self.text_widget.observe(self.on_text_change, names='value')
        self._updating = False

    def on_text_change(self, change):
        if self._updating:
            return

        self._updating = True
        current = change['new']

        # Remove existing commas for processing
        clean_value = current.replace(',', '')

        try:
            # Only format if it's a valid number
            num_value = float(clean_value)
            formatted = f"{num_value:,.2f}"
            if formatted != current:
                self.text_widget.value = formatted
        except ValueError:
            pass  # Keep the current value if invalid

        self._updating = False

    def get_value(self):
        """Return the numeric value without commas"""
        try:
            return float(self.text_widget.value.replace(',', ''))
        except ValueError:
            return 0.0

# Create a class to handle the GUI
class LifetimeRiskCalculator:
    def __init__(self):
        self.output = widgets.Output()

        # Create input widgets
        self.notional_input = FormattedTextInput('Notional:', 1000000)
        self.ticker = widgets.Text(
            value="AAPL",
            description='Ticker:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        self.date = widgets.DatePicker(
            value=dt.date(2025, 6, 10),
            description='Date:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        self.daily_volatility = FormattedTextInput('Daily Volatility (Optional):', 0.0)
        self.unwind_speed_as_pct_adv = FormattedTextInput('Unwind Speed as Pct Adv:', 0.03)

        # Create calculate button
        self.calculate_button = widgets.Button(
            description='Calculate Risk',
            button_style='primary',
            layout=widgets.Layout(width='300px')
        )
        self.calculate_button.on_click(self.on_calculate_clicked)

        # Create reset button
        self.reset_button = widgets.Button(
            description='Reset',
            button_style='warning',
            layout=widgets.Layout(width='300px')
        )
        self.reset_button.on_click(self.on_reset_clicked)

        # Display everything
        self.container = widgets.VBox([
            widgets.HTML(value="<h3>Lifetime Risk Calculator</h3>"),
            self.notional_input.text_widget,
            self.ticker,
            self.date,
            self.daily_volatility.text_widget,
            self.unwind_speed_as_pct_adv.text_widget,
            widgets.HBox([self.calculate_button, self.reset_button]),
            self.output
        ])

    def on_calculate_clicked(self, button):
        with self.output:
            clear_output()

            # Get values from inputs
            notional = self.notional_input.get_value()
            ticker = self.ticker.value
            date = self.date.value
            daily_volatility = self.daily_volatility.get_value()
            unwind_speed_as_pct_adv = self.unwind_speed_as_pct_adv.get_value()

            # Handle optional daily volatility
            daily_volatility = "Optional" if daily_volatility == 0.0 else daily_volatility

            # Calculate the result
            result, daily_vol = lifetime_risk_tcm_bps(notional, ticker, date, daily_volatility, unwind_speed_as_pct_adv)

            # Display the inputs and result
            data = {
                'Parameter': ['Notional', 'Ticker', 'Date', 'Daily Volatility', 'Unwind Speed as Pct Adv', 'Result (TCM BPS)'],
                'Value': [f"${notional:,.2f}", ticker, date, f"{daily_vol:.4f}", f"{unwind_speed_as_pct_adv:.2%}", f"{result:.2f}"]
            }

            result_df = pd.DataFrame(data)
            display(result_df)

    def on_reset_clicked(self, b):
        self.notional_input.text_widget.value = "1,000,000"
        self.ticker.value = "AAPL"
        self.date.value = dt.date(2025, 6, 10)
        self.daily_volatility.text_widget.value = "0.0"
        self.unwind_speed_as_pct_adv.text_widget.value = "0.03"

        with self.output:
            clear_output()
            print("Values reset to defaults.")

    def display(self):
        display(self.container)

# Create and display the calculator
calculator = LifetimeRiskCalculator()
calculator.display()